<a href="https://colab.research.google.com/github/erayyildiz/TurkishConversationBot/blob/master/LeylaMecnunChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install torch --upgrade 

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59cc8000 @  0x7fe6743882a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [4]:
!ls

sample_data


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
create_subtitle_utterences=False
if create_subtitle_utterences:
    !wget http://opus.nlpl.eu/download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr.gz
    !gunzip 'download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr.gz'
    !mv 'download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr' subtitles.txt
    previous_line = ""
    with open("subtitles.dialogues.txt", "w") as w:
        with open("subtitles.txt", "r") as f:
            for i, line in enumerate(f):
                line = line.strip()
                if line.startswith("-"):
                    line = line.replace("- ", "")
                    line = line.replace("-", "")
                    if previous_line:
                        w.write("{}\t{}\n".format(previous_line, line))
                    previous_line = line
                else:
                    previous_line = ""
    !mkdir "gdrive/My Drive/chatbot"
    !cp subtitles.dialogues.txt "gdrive/My Drive/chatbot/"

In [0]:
import os
import re
from nltk.tokenize import casual_tokenize

EOS = "<EOS>"
SOS = "<SOS>"

def to_lower(txt):
    txt = txt.replace('İ', 'i')
    txt = txt.replace('Ğ', 'ğ')
    txt = txt.replace('Ü', 'ü')
    txt = txt.replace('Ş', 'ş')
    txt = txt.replace('Ç', 'ç')
    txt = txt.replace('Ö', 'ö')
    return txt.lower()
    

utterance_regex = re.compile(r"^.*[a-zA-Z]+.*$")
nondiaolgue_regex = re.compile(r"\[[^\]]+\]")
nondiaolgue_regex2 = re.compile(r"<[^>]+>")
number_regex = re.compile(r"^[0-9]+$")

In [0]:
max_samples = 5000000
start_from = 0 
subtitle_utterances = []
with open("gdrive/My Drive/chatbot/subtitles.dialogues.txt", "r") as f:
    for i, line in enumerate(f):
        if i < start_from:
            continue
        parses = line.split("\t")
        q = to_lower(parses[0])
        a = to_lower(parses[1])
        q_tokens = [token for token in casual_tokenize(q) if utterance_regex.match(token)]
        a_tokens = [token for token in casual_tokenize(a) if utterance_regex.match(token)]
        subtitle_utterances.append((q_tokens, a_tokens))
        if i >= max_samples + start_from:
            break

In [9]:
len(subtitle_utterances)

5000001

In [0]:
def indexesFromSentence(sentence):
    return [word2int[word] for word in sentence if word in word2int]


def tensorFromSentence(sentence):
    indexes = indexesFromSentence(sentence)
    indexes.append(word2int[EOS])
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(pair[0])
    target_tensor = tensorFromSentence(pair[1])
    return (input_tensor, target_tensor)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
device

device(type='cuda')

In [0]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[word2int[SOS]]], device=device)

    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di] 

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, utterances, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(utterances))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        if input_tensor.shape[0] > 1 and target_tensor.shape[0] > 1:
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

In [0]:
def evaluate(encoder, decoder, sentence, max_length=20, beam_size=10):
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = casual_tokenize(to_lower(sentence))
        elif isinstance(sentence, list):
            tokens = sentence
        else:
            raise TypeError("Wrong Input Type: {}".format(type(sentence)))
        input_tensor = tensorFromSentence(tokens)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[word2int[SOS]]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == word2int[EOS]:
                decoded_words.append(EOS)
                break
            else:
                decoded_words.append(int2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [0]:
def evaluateRandomly(encoder, decoder, utterances, n=10):
    for i in range(n):
        pair = random.choice(utterances)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [18]:
import os.path
import pickle
ENCODER_PATH = "gdrive/My Drive/chatbot/chatbot.encoder.model"
DECODER_PATH = "gdrive/My Drive/chatbot/chatbot.decoder.model"
WORDS_PATH = "gdrive/My Drive/chatbot/chatbot.words"

if os.path.isfile(WORDS_PATH):
    print("Loading word list")
    with open(WORDS_PATH,"rb") as f:
        words = pickle.load(f)
        
else:
    word_counts_dic = {}
    for utterance in subtitle_utterances:
        for word in utterance[0]:
            if word not in word_counts_dic:
                word_counts_dic[word] = 1
            else:
                word_counts_dic[word] += 1

    min_word_count = 25
    words = [SOS, EOS]
    words += [w for w, c in word_counts_dic.items() if c >= min_word_count]
    with open(WORDS_PATH,"wb") as f:
        pickle.dump(words,f)

int2word = list(words)
word2int = {c:i for i,c in enumerate(words)}

Loading word list


In [19]:
len(words)

108606

In [0]:
hidden_size = 256
encoder1 = EncoderRNN(len(words), hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, len(words)).to(device)

In [21]:
import os

def load_model_states(model, path):
    """Load a previously saved model states."""
    # original saved file with DataParallel
    state_dict = torch.load(path)
    model.load_state_dict(state_dict)
        
if os.path.isfile(ENCODER_PATH):
    print("Loading Encoder")
    load_model_states(encoder1, ENCODER_PATH)
if os.path.isfile(DECODER_PATH):
    print("Loading Decoder")
    load_model_states(decoder1, DECODER_PATH)

Loading Encoder
Loading Decoder


In [0]:
leyla_mecnun_utterances = []
counter = 0
goon = False
for filename in os.listdir("gdrive/My Drive/Colab Notebooks/leyla ile mecnun data"):
    cur_utterance = ""
    previous_line = ""
    with open("gdrive/My Drive/Colab Notebooks/leyla ile mecnun data/{}".format(filename), "r", 
              encoding="ISO-8859-9") as f:
        print("Reading file: {}".format(filename))
        for line in f:
            line = to_lower(line).strip()
            if not utterance_regex.match(line) or line.startswith('#') or len(line) == 0:
                if len(cur_utterance) > 0 and len(previous_line):
                    previous_line_tokens = [token for token in casual_tokenize(previous_line) if utterance_regex.match(token)]
                    cur_utterance_tokens = [token for token in casual_tokenize(cur_utterance) if utterance_regex.match(token)]
                    leyla_mecnun_utterances.append((previous_line_tokens, cur_utterance_tokens))
                    cur_utterance = ""
                    previous_line = ""
                goon = False
                continue
              
            if line.startswith('-'):
                line = line.replace("-", "")
                line = nondiaolgue_regex.sub(r"", line).strip()
                line = nondiaolgue_regex2.sub(r"", line).strip()
                line = line.replace("#", "")
                previous_line = cur_utterance.strip()
                cur_utterance = line
                counter += 1
                goon = True
            elif goon:
              cur_utterance += " " + line
            
    

In [0]:
random.sample(subtitle_utterances, 10)

In [0]:
for epoch in range(10):
    trainIters(encoder1, decoder1, 100000, subtitle_utterances, print_every=10000, learning_rate=0.001)
    print("Epoch {} is completed.".format(epoch))
    evaluateRandomly(encoder1, decoder1, subtitle_utterances)
    torch.save(encoder1.state_dict(), ENCODER_PATH)
    torch.save(decoder1.state_dict(), DECODER_PATH)
    
    

In [39]:
for epoch in range(1):
    trainIters(encoder1, decoder1, 10000, leyla_mecnun_utterances, print_every=1000, learning_rate=0.001)
    print("Epoch {} is completed.".format(epoch))
    

1m 25s (- 12m 49s) (1000 10%) 4.0868
2m 50s (- 11m 20s) (2000 20%) 3.9567
4m 14s (- 9m 54s) (3000 30%) 3.7827
5m 39s (- 8m 28s) (4000 40%) 3.6652
7m 3s (- 7m 3s) (5000 50%) 3.7296
8m 28s (- 5m 38s) (6000 60%) 3.6377
9m 52s (- 4m 13s) (7000 70%) 3.4612
11m 14s (- 2m 48s) (8000 80%) 3.4275
12m 38s (- 1m 24s) (9000 90%) 3.5146
14m 3s (- 0m 0s) (10000 100%) 3.4253
Epoch 0 is completed.


In [0]:
from IPython.display import clear_output
import time


while True:
    input_txt = input()
    clear_output()
    output_words = evaluate(encoder1, decoder1, input_txt)
    output_sentence = ' '.join(output_words[:-1])
    print(" - {}".format(output_sentence))
    time.sleep(2)
    clear_output()